# Multilayer Perceptron


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Multilayer Perceptron is a type of artificial neural network that consists of multiple layers of interconnected nodes, called neurons. It is a feedforward neural network, meaning that the information flows in one direction, from the input layer to the output layer.

MLP is widely used in various machine learning tasks, such as classification, regression, and pattern recognition. It is known for its ability to learn complex patterns and make accurate predictions.


In [ ]:
# Plot the decision boundary of a classifier
# Not really important for the course
def plotgrid(f, dr=0):
    result = np.zeros((100, 100))
    for i in range(100):
        for j in range(100):
            x = np.array([i/100.0, j/100.0])
            z = f(x)
            if dr == 0:
                z /= sum(z)
                result[i, j] = z[1]
            else:
                result[i, j] = np.argmax(z)
    return result.T[::-1]

In [ ]:
def random_data():
    data = np.random.random_sample((100, 2))
    labels = (data[:, 0]-data[:, 1] < 0.3)

    d0 = data[labels == False]
    d1 = data[labels]

    targets = np.array([labels, 1-labels]).T
    return data, targets, labels, d0, d1

In [ ]:
data, targets, labels, d0, d1 = random_data()
plt.plot(d0[:, 0], d0[:, 1], "bo")
plt.plot(d1[:, 0], d1[:, 1], "ro")

## Fully Connected Feed-Forward Network

In [ ]:
class FCLayer:
    def __init__(self, output_size, input_size, activation=None):
        self.relu = activation == 'relu'
        self.sigmoid = activation == 'sigmoid'
        self.weights = np.random.randn(
            output_size, input_size) / np.sqrt(input_size)
        self.weight_update = np.zeros_like(self.weights)

    def forward(self, input):
        self.input = input.copy()
        self.y = np.dot(self.weights, self.input)
        if self.relu:
            self.y[self.y < 0] = 0
        if self.sigmoid:
            self.y = 1.0 / (1.0 + np.exp(-self.y))
        return self.y

    def backward(self, grad, _):
        if self.relu:
            grad[self.y <= 0] = 0
        if self.sigmoid:
            grad = grad * self.y * (1 - self.y)
        self.weight_update += np.outer(grad, self.input)
        return np.dot(self.weights.T, grad)

    def update_weights(self, learning_rate):
        self.weights -= learning_rate * self.weight_update
        self.weight_update = np.zeros_like(self.weights)


class Network:
    def __init__(self, topology, learning_rate):
        self.learning_rate = learning_rate
        self.topology = topology

    def update_weights(self):
        for layer in self.topology:
            layer.update_weights(self.learning_rate)

    def forward(self, x):
        for layer in self.topology:
            x = layer.forward(x)
        return x

    def backward(self, x, y):
        for layer in self.topology:
            x = layer.forward(x)
        for layer in reversed(self.topology):
            y = layer.backward(y, self.learning_rate)


network = Network(
    topology=[
        FCLayer(5, 2, activation='relu'),
        FCLayer(2, 5, activation='sigmoid')
    ],
    learning_rate=1)

In [ ]:
# Before training, the decision boundary is fuzzy and random
plt.imshow(plotgrid(network.forward, dr=0))

In [ ]:
for e in range(1000):
    i = np.random.randint(0, len(data))
    out = network.forward(data[i])
    # We divide by 2 to make the derivative of the loss easier
    # Loss = 1/2(target - output)^2
    # dLoss/dOutput = -(target - output)
    network.backward(data[i], -(targets[i]-out))
    network.update_weights()

correct = 0
for i in range(len(data)):
    out = np.argmax(network.forward(data[i]))
    if out == np.argmax(targets[i]):
        correct += 1

print("Correct predicted: ", 1.0 * correct /len(data))
plt.imshow(plotgrid(network.forward, dr=0))

## Tasks

Toy around with the parameters. See how the network behaves, if you adapt the activation functions.
Check what happens when you tweak the number of epochs.